In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import Tuple

In [ ]:
def rref_withpivot(A,r=A.shape[1]):
    # r is the number of columns that will be considered in the reduction phase
    U = np.copy(A)
    (m,n)=A.shape
    P = np.eye(m)
    j = 0
    for i in range(0,m): 
        ech=1
        while (ech == 1) & (j < r):
          indm=np.argmax(abs(U[i:m,j]))
          indm=indm+i
          if (indm != i) & (abs(U[indm,j]) > 1e-15): # perform the permutation
             U[ [i, indm],:]=U[[indm,i],:] 
             P[ [i, indm],:]=P[[indm,i],:] 
          if ( abs(U[i,j]) > 1e-15):
             M=U[i+1:m,j]/U[i,j]
             U[i+1:m,j+1:n]=U[i+1:m,j+1:n]-np.outer(M,U[i,j+1:n])
             U[i+1:m,j]=0           
             U[i,j:n] = U[i,j:n]/U[i,j]   # the pivotal element should be 1        
             if i>0:
               M= U[0:i,j]/U[i,j]
               U[0:(i),j:n]=U[0:i,j:n]-np.outer(M,U[i,j:n])     
             j=j+1
             ech=0
          else:
            j=j+1
            ech=1
            
    return(U,P)  

<details>
<summary>EXPLANATION</summary>

## 🔍 Comprehensive Analysis: Reduced Row Echelon Form with Partial Pivoting

### 📌 Algorithm Purpose & Mathematical Foundation

This function computes the **Reduced Row Echelon Form (RREF)** of a matrix $A \in \mathbb{R}^{m \times n}$ using **Gaussian elimination with partial pivoting**, while simultaneously tracking row permutations in a permutation matrix $P$. Unlike standard Gaussian elimination (which produces row echelon form), RREF enforces:

1. **Leading entries (pivots) equal to 1**
2. **Zeros above and below each pivot**
3. **Pivots move strictly rightward down rows**

Mathematically, the algorithm computes matrices $U$ and $P$ such that:
$$
PA = U \quad \text{where } U \text{ is in RREF}
$$
This decomposition is fundamental for:
- Solving linear systems $Ax = b$ via $Ux = Pb$
- Computing matrix rank
- Finding null spaces and bases for column/row spaces
- Determining linear independence of vectors

---

### 🔧 Variable Roles & Data Flow

| Variable | Role | Type | Significance |
|----------|------|------|--------------|
| `A` | Input matrix | `np.ndarray` | Original $m \times n$ matrix to reduce |
| `r` | Column limit | `int` | Max columns to process (default = full width $n$) |
| `U` | Working matrix | `np.ndarray` | Copy of $A$ transformed into RREF |
| `P` | Permutation matrix | `np.ndarray` | Tracks row swaps: $P = P_k \cdots P_1$ |
| `m, n` | Dimensions | `int` | $m$ = rows, $n$ = columns of $A$ |
| `i` | Row index | `int` | Current pivot row (0 to $m-1$) |
| `j` | Column index | `int` | Current pivot column (0 to $r-1$) |
| `ech` | Loop control flag | `int` | `1` = searching for pivot, `0` = pivot found |
| `indm` | Pivot row candidate | `int` | Index of max $|U[k,j]|$ for $k \geq i$ |
| `M` | Multiplier vector | `np.ndarray` | Elimination coefficients $M_k = U_{k,j}/U_{i,j}$ |

---

### 🧮 Line-by-Line Numerical Breakdown

#### Initialization Phase
```python
U = np.copy(A)
(m,n) = A.shape
P = np.eye(m)
j = 0
```
- **`np.copy(A)`**: Prevents mutation of input matrix (critical for numerical reproducibility)
- **`P = np.eye(m)`**: Identity matrix accumulates row swaps: each swap updates $P \leftarrow P_{\text{swap}} P$
- **`j = 0`**: Column pointer starts at leftmost column

#### Outer Loop: Pivot Row Selection
```python
for i in range(0, m):
    ech = 1
    while (ech == 1) & (j < r):
```
- Processes each row $i$ as a potential pivot row
- Inner `while` loop scans columns rightward until a valid pivot is found or columns exhausted
- **Numerical significance**: Handles rank-deficient matrices by skipping zero columns

#### Partial Pivoting Step
```python
indm = np.argmax(abs(U[i:m, j]))
indm = indm + i
if (indm != i) & (abs(U[indm, j]) > 1e-15):
    U[[i, indm], :] = U[[indm, i], :]
    P[[i, indm], :] = P[[indm, i], :]
```
- **`np.argmax(abs(U[i:m, j]))`**: Finds row with maximum absolute value in column $j$ below row $i$ → **partial pivoting**
- **Why?** Minimizes round-off error by avoiding division by small numbers (improves numerical stability)
- **Threshold `1e-15`**: Prevents pivoting on near-zero values (avoids amplifying floating-point noise)
- **Permutation tracking**: Swaps rows in both $U$ and $P$ to maintain $PA = U$ invariant

#### Pivot Validation & Elimination
```python
if (abs(U[i, j]) > 1e-15):
```
- **Numerical tolerance check**: Treats $|U_{i,j}| \leq 10^{-15}$ as zero (machine epsilon scale for double precision)
- Prevents division by near-zero values that would cause catastrophic cancellation

#### Forward Elimination (Below Pivot)
```python
M = U[i+1:m, j] / U[i, j]
U[i+1:m, j+1:n] = U[i+1:m, j+1:n] - np.outer(M, U[i, j+1:n])
U[i+1:m, j] = 0
```
- **`M = U[i+1:m, j] / U[i, j]`**: Computes multipliers $M_k = U_{k,j}/U_{i,j}$ for rows $k > i$
- **`np.outer(M, U[i, j+1:n])`**: Efficiently applies $M_k \cdot \text{row}_i$ to eliminate subdiagonal entries
- **`U[i+1:m, j] = 0`**: Explicitly zeros column below pivot (avoids residual floating-point noise)
- **Numerical significance**: Standard Gaussian elimination step; transforms matrix toward upper triangular form

#### Pivot Normalization
```python
U[i, j:n] = U[i, j:n] / U[i, j]
```
- Scales pivot row so $U_{i,j} = 1$ → satisfies RREF requirement
- **Caution**: Division amplifies relative error if $|U_{i,j}|$ is small (mitigated by pivoting)

#### Backward Elimination (Above Pivot)
```python
if i > 0:
    M = U[0:i, j] / U[i, j]
    U[0:i, j:n] = U[0:i, j:n] - np.outer(M, U[i, j:n])
```
- **Key RREF distinction**: Eliminates entries *above* pivot (not done in standard Gaussian elimination)
- Transforms upper triangular form → diagonal form with 1s on pivots
- **Numerical significance**: Completes reduction to canonical form required for null space computation

#### Column Advancement
```python
j = j + 1
ech = 0  # Exit while loop after successful pivot
```
- Moves to next column after pivot processing
- `ech = 0` breaks inner loop to proceed to next row

#### Zero Column Handling
```python
else:
    j = j + 1
    ech = 1  # Continue searching in next column
```
- Skips columns with no valid pivot (rank deficiency)
- Critical for handling singular matrices and computing true rank

---

### ⚠️ Edge Cases & Stability Considerations

| Edge Case | Handling | Numerical Risk |
|-----------|----------|----------------|
| **Singular matrix** | Skips zero columns via `else` branch; returns RREF with zero rows | Correct rank detection |
| **Near-singular matrix** | Threshold `1e-15` avoids false pivots | Risk of misclassifying ill-conditioned systems |
| **Rectangular matrix** (`m ≠ n`) | Works for any $m \times n$; `r` parameter limits columns | None |
| **All-zero rows** | Naturally emerge as trailing zero rows in RREF | Correct representation |
| **Floating-point noise** | Explicit zeroing (`U[i+1:m,j]=0`) suppresses residuals | Prevents spurious pivots |
| **Rank deficiency** | `j` may not reach `r`; final `j` = matrix rank | Accurate rank computation |

**Stability notes**:
- ✅ **Partial pivoting** reduces growth factor in elimination (backward error $\mathcal{O}(\epsilon_{\text{mach}} \kappa(A))$)
- ⚠️ **No scaling**: Column scaling could further improve stability for badly scaled matrices
- ⚠️ **Fixed tolerance**: `1e-15` may be inappropriate for very large/small magnitude problems (adaptive tolerance preferred)

---

### ⏱️ Computational Complexity

| Operation | Cost per Pivot | Total Cost |
|-----------|----------------|------------|
| Pivot search | $\mathcal{O}(m-i)$ | $\mathcal{O}(mn)$ |
| Row swap | $\mathcal{O}(n)$ | $\mathcal{O}(mn)$ |
| Forward elimination | $\mathcal{O}((m-i)n)$ | $\mathcal{O}(m^2n)$ |
| Backward elimination | $\mathcal{O}(in)$ | $\mathcal{O}(m^2n)$ |
| **Total** | — | **$\mathcal{O}(m^2n)$** |

- Dominated by elimination steps (similar to standard Gaussian elimination)
- RREF requires extra backward pass → ~2× cost of row echelon form
- Memory complexity: $\mathcal{O}(mn)$ for $U$ and $P$

---

### 💡 Practical Usage Examples

```python
import numpy as np

# Example 1: Solve linear system Ax = b
A = np.array([[2, 1, -1],
              [-3, -1, 2],
              [-2, 1, 2]], dtype=float)
b = np.array([[8], [-11], [-3]], dtype=float)
Ab = np.hstack([A, b])  # Augmented matrix

U, P = rref_withpivot(Ab)
# Solution in last column of U (if consistent)
x = U[:, -1][:3]  # Extract solution vector

# Example 2: Compute matrix rank
A = np.array([[1, 2, 3],
              [2, 4, 6],
              [1, 0, 1]], dtype=float)
U, _ = rref_withpivot(A)
rank = np.sum(np.abs(np.diag(U[:3,:3])) > 1e-15)  # Count non-zero pivots

# Example 3: Find null space basis
A = np.array([[1, 2, 3],
              [2, 4, 6]], dtype=float)
U, _ = rref_withpivot(A)
# Free variables correspond to non-pivot columns → construct basis vectors
```

---

### 📚 Pedagogical Notes for Numerical Analysis Students

- **Why pivoting matters**: Without pivoting, elimination on $\begin{bmatrix} \epsilon & 1 \\ 1 & 1 \end{bmatrix}$ with $\epsilon \approx 10^{-20}$ causes catastrophic cancellation. Pivoting swaps rows to avoid division by $\epsilon$.
  
- **RREF vs REF**: Standard Gaussian elimination stops at row echelon form (REF). RREF adds backward elimination and normalization → unique canonical form essential for theoretical analysis.

- **Tolerance selection**: The `1e-15` threshold assumes:
  - Double-precision arithmetic ($\epsilon_{\text{mach}} \approx 2.2 \times 10^{-16}$)
  - Moderately scaled problems ($\|A\| \sim \mathcal{O}(1)$)
  For ill-scaled problems, use relative tolerance: `tol = 1e-12 * np.max(np.abs(U[:,j]))`

- **Permutation matrix utility**: $P$ enables reconstruction of original row operations. Critical when solving $Ax=b$: solution satisfies $Ux = Pb$, not $Ux = b$.

- **Limitation**: This implementation does **not** handle symbolic computation or exact arithmetic. For rational matrices, use exact RREF algorithms (e.g., fraction arithmetic) to avoid floating-point artifacts.

This algorithm provides a numerically robust foundation for linear algebra computations while illustrating core principles of stability, pivoting, and canonical forms essential to numerical analysis.
</details>

In [ ]:
def rref_with_pivoting_verbose(
    input_matrix: np.ndarray,
    reduction_columns: int = None,
    tolerance: float = 1e-15,
    verbose: bool = True
) -> Tuple[np.ndarray, np.ndarray, dict]:
    """
    Compute Reduced Row Echelon Form (RREF) with partial pivoting.
    
    This implementation follows Gaussian-Jordan elimination with:
    - Partial pivoting (row swaps for numerical stability)
    - Forward elimination to upper triangular form
    - Backward elimination to reduced form
    - Explicit logging of all row operations
    
    Parameters
    ----------
    input_matrix : np.ndarray
        Input matrix (m x n) to reduce
    reduction_columns : int, optional
        Number of columns to process (default: all columns)
    tolerance : float
        Threshold below which values are considered zero
    verbose : bool
        Whether to print step-by-step operations
    
    Returns
    -------
    reduced_matrix : np.ndarray
        Matrix in reduced row echelon form
    permutation_matrix : np.ndarray
        Cumulative row permutation matrix (P such that P·A = RREF)
    metadata : dict
        Diagnostic information including:
        - pivot_positions: list of (row, col) pivot locations
        - row_swaps: count of row exchanges
        - rank: computed matrix rank
        - operations_log: detailed operation history
    """
    # ===== ERROR HANDLING =====
    if not isinstance(input_matrix, np.ndarray):
        raise TypeError("Input must be a NumPy ndarray")
    if input_matrix.ndim != 2:
        raise ValueError("Input matrix must be 2-dimensional")
    if input_matrix.size == 0:
        raise ValueError("Input matrix cannot be empty")
    if not np.issubdtype(input_matrix.dtype, np.number):
        raise TypeError("Matrix must contain numeric values only")
    
    # Create working copies to avoid modifying original data
    working_matrix = np.copy(input_matrix).astype(float)
    num_rows, num_cols = working_matrix.shape
    
    # Default to processing all columns if not specified
    if reduction_columns is None:
        reduction_columns = num_cols
    elif reduction_columns < 0 or reduction_columns > num_cols:
        raise ValueError(f"reduction_columns must be between 0 and {num_cols}")
    
    # Initialize permutation matrix (identity)
    permutation_matrix = np.eye(num_rows)
    
    # Diagnostic tracking
    pivot_positions = []
    row_swap_count = 0
    operations_log = []
    
    current_row = 0  # Row index for next pivot
    
    if verbose:
        print(f"{'='*60}")
        print(f"RREF COMPUTATION WITH PARTIAL PIVOTING")
        print(f"{'='*60}")
        print(f"Input matrix shape: {num_rows} × {num_cols}")
        print(f"Columns to reduce: {reduction_columns}")
        print(f"Tolerance: {tolerance:.2e}\n")
        print("Initial matrix:")
        print(working_matrix)
        print()
    
    # ===== MAIN ELIMINATION LOOP =====
    for current_col in range(reduction_columns):
        # Terminate early if we've exhausted all rows
        if current_row >= num_rows:
            if verbose:
                operations_log.append(f"Terminating early: all rows processed (row {current_row} ≥ {num_rows})")
                print(operations_log[-1])
            break
        
        # ===== STEP 1: PARTIAL PIVOTING =====
        # Find row with maximum absolute value in current column (from current_row downward)
        column_segment = np.abs(working_matrix[current_row:, current_col])
        max_index_in_segment = np.argmax(column_segment)
        pivot_row = current_row + max_index_in_segment
        pivot_value = working_matrix[pivot_row, current_col]
        
        # Log pivot selection
        log_msg = f"Column {current_col}: max |value| = {np.max(column_segment):.4e} at row {pivot_row} (segment index {max_index_in_segment})"
        operations_log.append(log_msg)
        if verbose:
            print(log_msg)
        
        # ===== STEP 2: ROW SWAP IF NEEDED =====
        if pivot_row != current_row and abs(pivot_value) > tolerance:
            # Swap rows in working matrix
            working_matrix[[current_row, pivot_row], :] = working_matrix[[pivot_row, current_row], :]
            # Track swap in permutation matrix
            permutation_matrix[[current_row, pivot_row], :] = permutation_matrix[[pivot_row, current_row], :]
            row_swap_count += 1
            
            log_msg = f"  → Swapped rows {current_row} ↔ {pivot_row}"
            operations_log.append(log_msg)
            if verbose:
                print(log_msg)
                print(f"     Matrix after swap:\n{working_matrix}")
        
        # ===== STEP 3: CHECK FOR ZERO COLUMN (no pivot possible) =====
        current_pivot_value = working_matrix[current_row, current_col]
        if abs(current_pivot_value) <= tolerance:
            log_msg = f"  → Column {current_col} is numerically zero (|pivot| = {abs(current_pivot_value):.2e} ≤ {tolerance:.2e}). Skipping column."
            operations_log.append(log_msg)
            if verbose:
                print(log_msg)
            continue  # Move to next column without advancing row counter
        
        # ===== STEP 4: NORMALIZE PIVOT ROW =====
        # Scale pivot row so pivot element becomes exactly 1
        scaling_factor = current_pivot_value
        working_matrix[current_row, current_col:] /= scaling_factor
        
        log_msg = f"  → Normalized row {current_row}: divided by {scaling_factor:.4e}"
        operations_log.append(log_msg)
        if verbose:
            print(log_msg)
            print(f"     Row {current_row} after normalization: {working_matrix[current_row, :]}")
        
        # ===== STEP 5: ELIMINATE ALL OTHER ENTRIES IN PIVOT COLUMN =====
        # Process rows ABOVE current pivot (backward elimination)
        for target_row in range(current_row):
            multiplier = working_matrix[target_row, current_col]
            if abs(multiplier) > tolerance:
                working_matrix[target_row, current_col:] -= multiplier * working_matrix[current_row, current_col:]
                log_msg = f"  → Eliminated row {target_row}: subtracted {multiplier:.4e} × row {current_row}"
                operations_log.append(log_msg)
                if verbose:
                    print(log_msg)
        
        # Process rows BELOW current pivot (forward elimination)
        for target_row in range(current_row + 1, num_rows):
            multiplier = working_matrix[target_row, current_col]
            if abs(multiplier) > tolerance:
                working_matrix[target_row, current_col:] -= multiplier * working_matrix[current_row, current_col:]
                log_msg = f"  → Eliminated row {target_row}: subtracted {multiplier:.4e} × row {current_row}"
                operations_log.append(log_msg)
                if verbose:
                    print(log_msg)
        
        # Record successful pivot
        pivot_positions.append((current_row, current_col))
        log_msg = f"✓ Pivot established at ({current_row}, {current_col}) = {working_matrix[current_row, current_col]:.4e}"
        operations_log.append(log_msg)
        if verbose:
            print(log_msg)
            print(f"\nMatrix after processing column {current_col}:")
            print(working_matrix)
            print()
        
        # Advance to next row for next pivot
        current_row += 1
    
    # ===== POST-PROCESSING: NUMERICAL CLEANUP =====
    # Zero out near-zero values for cleaner output
    working_matrix[np.abs(working_matrix) < tolerance] = 0.0
    
    # Compute rank from pivot count
    computed_rank = len(pivot_positions)
    
    # Prepare metadata
    metadata = {
        'pivot_positions': pivot_positions,
        'row_swaps': row_swap_count,
        'rank': computed_rank,
        'operations_log': operations_log,
        'tolerance': tolerance
    }
    
    if verbose:
        print(f"{'='*60}")
        print(f"RREF COMPUTATION COMPLETE")
        print(f"{'='*60}")
        print(f"Final rank: {computed_rank}")
        print(f"Total row swaps: {row_swap_count}")
        print(f"Pivot positions: {pivot_positions}")
        print(f"\nReduced matrix (RREF):")
        print(working_matrix)
        print(f"\nPermutation matrix P (such that P·A ≈ RREF):")
        print(permutation_matrix)
        print(f"{'='*60}\n")
    
    return working_matrix, permutation_matrix, metadata

In [ ]:
'''Solve the linear system:
x + 2y = 5
3x + 4y = 6
Augmented matrix: [[1, 2, 5], [3, 4, 6]] → Expected solution: x = -4, y = 4.5'''

def example_textbook():
    print("\n" + "="*70)
    print("EXAMPLE 1: TEXTBOOK VERIFICATION (2×3 AUGMENTED SYSTEM)")
    print("="*70)
    
    # Define augmented matrix [A|b]
    A_aug = np.array([[1, 2, 5],
                      [3, 4, 6]], dtype=float)
    
    print("\nSystem:")
    print("  x + 2y = 5")
    print("  3x + 4y = 6")
    print("\nAugmented matrix [A|b]:")
    print(A_aug)
    
    # Compute RREF
    R, P, meta = rref_with_pivoting_verbose(A_aug, verbose=False)
    
    # Extract solution
    x_sol = R[0, -1]
    y_sol = R[1, -1]
    
    # Analytical solution for verification
    x_exact = -4.0
    y_exact = 4.5
    
    # Error analysis
    abs_err_x = abs(x_sol - x_exact)
    abs_err_y = abs(y_sol - y_exact)
    residual = np.linalg.norm(A_aug[:, :-1] @ np.array([x_sol, y_sol]) - A_aug[:, -1])
    
    print("\nComputed RREF:")
    print(R)
    print(f"\nSolution: x = {x_sol:.6f}, y = {y_sol:.6f}")
    print(f"Exact solution: x = {x_exact}, y = {y_exact}")
    print(f"\nError analysis:")
    print(f"  |x_computed - x_exact| = {abs_err_x:.2e}")
    print(f"  |y_computed - y_exact| = {abs_err_y:.2e}")
    print(f"  Residual norm ‖Ax - b‖ = {residual:.2e}")
    
    # Visualization: Convergence of pivot values
    pivot_vals = [abs(R[i, j]) for i, j in meta['pivot_positions']]
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.spy(R, markersize=15, color='blue')
    plt.title('RREF Sparsity Pattern\n(Non-zero elements)')
    plt.xlabel('Column index')
    plt.ylabel('Row index')
    
    plt.subplot(1, 2, 2)
    plt.plot(range(len(pivot_vals)), pivot_vals, 'ro-', linewidth=2, markersize=8)
    plt.axhline(y=1.0, color='g', linestyle='--', label='Ideal pivot = 1')
    plt.title('Pivot Magnitudes During Elimination')
    plt.xlabel('Pivot step')
    plt.ylabel('Absolute pivot value')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('example1_rref_analysis.png', dpi=150, bbox_inches='tight')
    print("\n✓ Visualization saved: example1_rref_analysis.png")
    plt.close()
    
    return R, meta